In [ ]:
from os import listdir,makedirs
from os.path import isfile,join
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import pickle
from PIL import Image
import os.path, sys
from keras.applications import vgg16
from keras import Model
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
def k_means_clustering(K, path, sub_path):
    
    for item in os.listdir(path):
        fullpath = os.path.join(path,item)
        pathos = os.path.join(sub_path,item)
        if os.path.isfile(fullpath):
            img = np.array(Image.open(fullpath))
            f, e = os.path.splitext(pathos)
            Z = img.reshape((-1,3))

            # convert to np.float32
            Z = np.float32(Z)

            # define criteria, number of clusters(K) and apply kmeans()
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
            ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
            
            # Now convert back into uint8, and make original image
            center = np.uint8(center)
            res = center[label.flatten()]
            res2 = res.reshape((img.shape))

            Image.fromarray(res2).save(f + 'kmeans.png', "png", quality=100)     
 

In [ ]:
path = "../test"
sub_path = "../result"
k_means_clustering(3, path, sub_path)

In [ ]:
###########################################################################################################################

In [ ]:
# Inputs image to the VGG16 model and gets image features
def ef(img):
    base_model = vgg16.VGG16(weights='imagenet', include_top=True)
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = vgg16.preprocess_input(img_data)
    img_data = np.asarray(img_data)
    # Predict
    vgg16_feature = model.predict(img_data)
    return vgg16_feature[0]

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

files = os.listdir("../test")
images = np.zeros((len(files),7*7*512))
for i in range(len(files)):
    img = image.load_img(os.path.join("../test",files[i]), target_size=(224, 224))
    feature = ef(img)
    images[i] = feature.flatten()

In [ ]:
print(images.shape)
print(images)
kmeans = KMeans(n_clusters=4, random_state=0).fit(images)

In [ ]:
labels = kmeans.labels_
labels = np.reshape(labels, (int(len(labels)/9),9))
print(labels)